# VeBiDraCor – A very big Drama Corpus

Workflow to create a very big Drama Corpus from DraCor plays

Use the supplied `docker-compose.empty.yml` to setup a local instance of the DraCor-Platform: `docker-compose -f docker-compose.empty.yml up`
The local platform uses the pre-built images defined in the docker-compose file:

* https://hub.docker.com/repository/docker/ingoboerner/dracor-api (tag: v0.86.2_local) is based on the offical [v0.86.0](https://github.com/dracor-org/dracor-api/releases/tag/v0.86.0) with added support for github actions to build the image)
* https://hub.docker.com/repository/docker/ingoboerner/dracor-frontend (tag: v1.4.3_local) is based on the official [v.1.4.0](https://github.com/dracor-org/dracor-frontend/releases/tag/v1.4.0) with support for github action to build the image)
* https://hub.docker.com/repository/docker/ingoboerner/dracor-metrics (tag: v1.2.1) is based on the official [v1.2.0](https://github.com/dracor-org/dracor-metrics/releases/tag/v1.2.0) with support for github action to build the image)

Check http://localhost:8088/api/info. Should return:

```
{
  "name" : "DraCor API",
  "version" : "0.86.2_local",
  "status" : "beta",
  "existdb" : "6.0.1"
}
```

http://localhost:8088/api/corpora should be empty.

After the empty platform is set-up, we proceed with creating vebidracor.

From docker-notebook, a function to send `GET` requests to the new instance

In [1]:
# import libraries json and requests
import json
import requests

#corpusname:str -> []
def get(**kwargs):
    #corpusname=corpusname
    #playname=playname
    #apibase="https://dracor.org/api/"
    #method=method
    #parse_json: True
    
    #could set different apibase, e.g. https://staging.dracor.org/api/ [not recommended, pls use the production server]
    if "apibase" in kwargs:
        if kwargs["apibase"].endswith("/"):
            apibase = kwargs["apibase"]
        else:
            apibase = kwargs["apibase"] + "/"
    else:
        #use local API per default
        apibase = "http://localhost:8088/api/"
    if "corpusname" in kwargs and "playname" in kwargs:
        # used for /api/corpora/{corpusname}/play/{playname}/
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"]
    elif "corpusname" in kwargs and not "playname" in kwargs:
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"] 
    elif "method" in kwargs and not "corpusname" in kwargs and not "playname" in kwargs:
            request_url = apibase + kwargs["method"]
            
    else: 
        #nothing set
        request = request_url = apibase + "info"
    
    #send the response
    r = requests.get(request_url)
    if r.status_code == 200:
        #success!
        if "parse_json" in kwargs:
            if kwargs["parse_json"] == True:
                json_data = json.loads(r.text)
                return json_data
            else:
                return r.text
        else:
            return r.text
    else:
        raise Exception("Request was not successful. Server returned status code: "  + str(r.status_code))

In [2]:
get(parse_json=True)

{'name': 'DraCor API',
 'version': '0.86.2_local',
 'status': 'beta',
 'existdb': '6.0.1'}

## Setting up the corpus

Running instance is empty.

In [3]:
get(method="corpora", parse_json=True)

[{'uri': 'https://dracor.org/api/corpora/vebi',
  'title': 'Very Big Drama Corpus',
  'name': 'vebi',
  'acronym': 'VebiDraCor'}]

Will set-up an empty corpus with corpusname `vebi`.

In [4]:
vebi_corpusname = "vebi"

#needed for authorization
from requests.auth import HTTPBasicAuth

#Username of the local instance
usr = "admin"
#Password of the admin user
pwd = ""

#construct the payload
vebi_metadata = {
  "name": "vebi",
  "title": "Very Big Drama Corpus"
}

#url of the corpora endpoint
corpora_endpoint_url = "http://localhost:8088/api/corpora"

#send the POST request using library requests
r = requests.post(corpora_endpoint_url, json = vebi_metadata, auth=HTTPBasicAuth(usr, pwd))

if r.status_code == 200:
    print("Success!" + " http://localhost:8088/vebi")

In [5]:
get(method="corpora", parse_json=True)

[{'uri': 'https://dracor.org/api/corpora/vebi',
  'title': 'Very Big Drama Corpus',
  'name': 'vebi',
  'acronym': 'VebiDraCor'}]

## Plays to load

Peer:

> 've been looking again at which corpora we should include in our VeBiDraCor (Very Big Drama Corpus) and I think: actually all of them, except the German Shakespeare corpus (gersh). And in GerDraCor we should not consider ger000480.
 
**DraCor corpora**
* als
* bash
* cal
* fre
* ger (exl. ger000480)
* greek
* hun
* ita
* rom
* rus
* shake
* span
* swe
* tat

also some plays from fredracor were excluded in this version:

* fre000038
* fre000057
* fre000065
* fre000099

We use a fork of shakedracor https://github.com/ingoboerner/shakedracor because the large original shakedracor at the time beeing caused problems.


We create a list of dictionaries, that hold the info on the current version, referenced by commit-id, of the corpora to use:

In [6]:
#list of corpora to include
corpora_to_include = [
    { 
        "corpusname": "als",
        "repository": "https://github.com/dracor-org/alsdracor",
        "commit" : "c87ea41aac9412e4bd84a28e9c7632c53904f77c"
    },
    { 
        "corpusname": "bash",
        "repository" : "https://github.com/dracor-org/bashdracor",
        "commit" : "c16b58ef3726a63c431bb9575b682c165c9c0cbd"
    },
    { 
        "corpusname": "cal",
        "repository": "https://github.com/dracor-org/caldracor",
        "commit": "6cb804d415051d5f18bc4841fa1ce4343a7f0ab5"
    },
    { 
        "corpusname": "fre",
        "repository": "https://github.com/dracor-org/fredracor",
        "commit": "65e93f6ff632b367cdc7e16e3e390956856c4b98",
        "exclude" : ["fre000038", "fre000057", "fre000065", "fre000099"]
    },
    { 
        "corpusname": "ger",
        "repository": "https://github.com/dracor-org/gerdracor",
        "commit": "9135bd4598f54133f23df6edfc983b79f1616fb5",
        "exclude" : ["ger000480"]
    },
    { 
        "corpusname": "greek",
        "repository": "https://github.com/dracor-org/greekdracor",
        "commit": "7397aafa1927c3e0a0720bf3c00bf367ab679f26"
    },
    { 
        "corpusname": "hun",
        "repository": "https://github.com/dracor-org/hundracor",
        "commit": "57e64454a73ffd984ff5fcc1c9b7bc16f3a169f2"
    },
    { 
        "corpusname": "ita",
        "repository": "https://github.com/dracor-org/itadracor",
        "commit": "10c84b416d25a6cbfbb195b9f82f136e482a7093"
    },
    { 
        "corpusname": "rom",
        "repository": "https://github.com/dracor-org/romdracor",
        "commit": "20644eb44f59649721310c3a6d1fd1fe505653d5"
    },
    { 
        "corpusname": "rus",
        "repository": "https://github.com/dracor-org/rusdracor",
        "commit": "6d5b1e5549731538a48684a456006384da206e9a"
    },
    { 
        "corpusname": "shake",
        "repository": "https://github.com/ingoboerner/shakedracor",
        "commit" : "3a420de7d253a505d1d3b8225e6bb6659577d82f"
    },
    { 
        "corpusname": "span",
        "repository": "https://github.com/dracor-org/spandracor",
        "commit": "184ebf975ad9cd674ff37cab44a181fa7ed8d85f"
    },
    { 
        "corpusname": "swe",
        "repository": "https://github.com/dracor-org/swedracor",
        "commit": "0e73db9315c9c8ed64abff7d2053f84e76fcf7ec"
    },
    { 
        "corpusname": "tat",
        "repository": "https://github.com/dracor-org/tatdracor",
        "commit": "5c71364f39f6533baa3a2e04217fd39e0898c851"
    }
    ]

### Load the data from github to the local instance

In [7]:
def load_play(corpus_info:dict, playname:str):
    """
    Load a single play to the corpus
    expects the corpus dictionary from corpora_to_include
    """
    
    headers = {'Content-Type': 'application/xml'}

    #define the variables
    #have to change this to be able to get dracor/ingoboerner shakedracor
    if "/dracor-org/" in corpus_info["repository"]:
        corpus_repo_part = corpus_info["repository"].split("/dracor-org/")[1] # this is not the same as "corpusname"!
    elif "/ingoboerner/" in corpus_info["repository"]:
        corpus_repo_part = corpus_info["repository"].split("/ingoboerner/")[1] # this is not the same as "corpusname"!
    else:
        raise Exception("Unexpected source repo.")
    
    commit_id = corpus_info["commit"]
    filename = playname + ".xml"

    #concatinate to a download url
    if "/dracor-org/" in corpus_info["repository"]:
        download_url = "https://raw.githubusercontent.com/dracor-org/" + corpus_repo_part + "/" + commit_id + "/tei/" + filename
    elif "/ingoboerner/" in corpus_info["repository"]:
        download_url = "https://raw.githubusercontent.com/ingoboerner/" + corpus_repo_part + "/" + commit_id + "/tei/" + filename
    else:
        raise Exception("Unexpected source repo when creating download-url.")
    
    
    get_r = requests.get(download_url)
    #get only the text from the response and encode it in UTF-8 (important!)
    if get_r.status_code == 200:
        #successful
        tei = get_r.text.encode('utf-8')
        
        #construct the URL to use in the PUT request:
        put_request_url = "http://localhost:8088/api/corpora/" + vebi_corpusname + "/play/" + playname + "/tei"
        
        put_r = requests.put(put_request_url, data=tei, headers=headers, auth=HTTPBasicAuth(usr, pwd))
        
        return put_r.status_code #should be 200 if successful
    else: 
        return 480
        


In [8]:
#actually, this should look into the files that are int /tei in the commit making use of the github API, not check on Dracor
#currently, I assume, that plays are added, but not deleted
def load_corpus(corpus_info:dict):
    
    #use DraCor API
    plays = get(apibase="https://dracor.org/api/", corpusname=corpus_info["corpusname"], parse_json=True)["dramas"]
    cnt_plays = len(plays)
    print("Loading " + corpus_info["corpusname"] + " " + str(cnt_plays) )
    
    if "exclude" in corpus_info:
        exclude = corpus_info["exclude"]
    else:
        exclude = []
    
    #errors
    errors = []
    
    n = 1
    for play in plays:
        playname = play["name"] #playname
        play_id = play["id"]
        #use the other function with the playname
        
        #check, if play should be excluded
        if play_id not in exclude:
            l = load_play(corpus_info, playname)
            if l != 200:
                error = {}
                error["playname"] = playname
                error["status_code"] = l
                errors.append(error)
            else:
                print("stored " + playname + " (" + str(n) + "/" + str(cnt_plays) + ")")
        
        else:
            #play should be excluded
            print("exluded " + playname)
        
        n=n+1
            

In [9]:
corpora_to_include

[{'corpusname': 'als',
  'repository': 'https://github.com/dracor-org/alsdracor',
  'commit': 'c87ea41aac9412e4bd84a28e9c7632c53904f77c'},
 {'corpusname': 'bash',
  'repository': 'https://github.com/dracor-org/bashdracor',
  'commit': 'c16b58ef3726a63c431bb9575b682c165c9c0cbd'},
 {'corpusname': 'cal',
  'repository': 'https://github.com/dracor-org/caldracor',
  'commit': '6cb804d415051d5f18bc4841fa1ce4343a7f0ab5'},
 {'corpusname': 'fre',
  'repository': 'https://github.com/dracor-org/fredracor',
  'commit': '65e93f6ff632b367cdc7e16e3e390956856c4b98',
  'exclude': ['fre000038', 'fre000057', 'fre000065', 'fre000099']},
 {'corpusname': 'ger',
  'repository': 'https://github.com/dracor-org/gerdracor',
  'commit': '9135bd4598f54133f23df6edfc983b79f1616fb5',
  'exclude': ['ger000480']},
 {'corpusname': 'greek',
  'repository': 'https://github.com/dracor-org/greekdracor',
  'commit': '7397aafa1927c3e0a0720bf3c00bf367ab679f26'},
 {'corpusname': 'hun',
  'repository': 'https://github.com/dracor

In [10]:
%%time
#load the data
for corpus in corpora_to_include:
    load_corpus(corpus)

Loading als 25
stored arnold-der-pfingstmontag (1/25)
stored bastian-dr-hans-im-schnokeloch (2/25)
stored bastian-dr-hofnarr-heidideldum (3/25)
stored bastian-dr-maischter-hett-gewunne (4/25)
stored bastian-e-sportschochzitt (5/25)
stored clemens-a-latzi-visit (6/25)
stored clemens-charlot (7/25)
stored clemens-chrischtowe (8/25)
stored clemens-gift (9/25)
stored greber-d-jumpfer-prinzesse (10/25)
stored greber-lucie (11/25)
stored greber-sainte-cecile (12/25)
stored hart-dr-poetisch-oscar (13/25)
stored horsch-d-madam-fahrt-velo (14/25)
stored jost-daa-im-narrehuss (15/25)
stored jost-so-e-liederlicher-frack (16/25)
stored kettner-uff-dr-hochzittsreis (17/25)
stored kuehne-bureaukrate (18/25)
stored riff-s-paradies (19/25)
stored riff-sainte-barbe (20/25)
stored schneegans-steckelburjer-universite-saaue (21/25)
stored stoskopf-dr-herr-maire (22/25)
stored stoskopf-dr-hoflieferant (23/25)
stored stoskopf-ins-ropfers-apothek (24/25)
stored weber-yo-yo (25/25)
Loading bash 3
stored karim

stored dicha-y-desdicha-del-nombre (172/205)
stored loa-a-el-divino-orfeo (173/205)
stored loa-a-el-ano-santo-de-roma (174/205)
stored de-una-causa-dos-efectos (175/205)
stored darlo-todo-y-no-dar-nada (176/205)
stored dar-tiempo-al-tiempo (177/205)
stored cual-es-mayor-perfeccion-hermosura-o-discrecion (178/205)
stored con-quien-vengo-vengo (179/205)
stored como-se-comunican-dos-estrellas-contrarias (180/205)
stored celos-aun-del-aire-matan (181/205)
stored cefalo-y-pocris (182/205)
stored casa-con-dos-puertas-mala-es-de-guardar (183/205)
stored cada-uno-para-si (184/205)
stored bien-vengas-mal-si-vienes-solo (185/205)
stored basta-callar (186/205)
stored auristela-y-lisidante (187/205)
stored argenis-y-poliarco (188/205)
stored apolo-y-climene (189/205)
stored antes-que-todo-es-mi-dama (190/205)
stored andromeda-y-perseo (191/205)
stored amor-honor-y-poder (192/205)
stored amigo-amante-y-leal (193/205)
stored amar-y-ser-amado-y-divina-filotea (194/205)
stored amar-despues-de-la-muert

stored boursault-fete-de-la-seine (165/1560)
stored boursault-germanicus (166/1560)
stored boursault-jaloux-endormi (167/1560)
stored boursault-marie-stuard (168/1560)
stored boursault-medecin-volant (169/1560)
stored boursault-mort-vivant (170/1560)
stored boursault-mots-a-la-mode (171/1560)
stored boursault-portrait-du-peintre (172/1560)
stored boursault-satire-des-satires (173/1560)
stored boyera-caton (174/1560)
stored boyer-agamemnon (175/1560)
stored boyer-amours-jupiter-semele (176/1560)
stored boyer-aristodeme (177/1560)
stored boyer-artaxerce (178/1560)
stored boyer-clotilde (179/1560)
stored boyer-federic (180/1560)
stored boyer-jeune-marius (181/1560)
stored boyer-judith (182/1560)
stored boyer-lisimene (183/1560)
stored boyer-mort-enfants-brute (184/1560)
stored boyer-oropaste (185/1560)
stored boyer-porcie-romaine (186/1560)
stored boyer-porus (187/1560)
stored boyer-tiridate (188/1560)
stored brohan-compter-sans-son-hote (189/1560)
stored brohan-qui-femme-a-guerre-a (190/

stored corneillep-rodogune (360/1560)
stored corneillep-sertorius (361/1560)
stored corneillep-sophonisbe (362/1560)
stored corneillep-suite-menteur (363/1560)
stored corneillep-suivante (364/1560)
stored corneillep-surena (365/1560)
stored corneillep-theodore (366/1560)
stored corneillep-tite (367/1560)
stored corneillep-toison (368/1560)
stored corneillep-veuve-34 (369/1560)
stored corneillep-veuve (370/1560)
stored dupaty-luce-salverte-bievre (371/1560)
stored corneillet-amour-a-la-mode (372/1560)
stored corneillet-antiochus (373/1560)
stored corneillet-ariane (374/1560)
stored corneillet-baron-d-albikrac (375/1560)
stored corneillet-berenice (376/1560)
stored corneillet-berger-extravagant (377/1560)
stored corneillet-bradamante (378/1560)
stored corneillet-camma (379/1560)
stored corneillet-charme-de-la-voix (380/1560)
stored corneillet-circe (381/1560)
stored corneillet-comte-d-essex (382/1560)
stored corneillet-comtesse-orgueil (383/1560)
stored corneillet-darius (384/1560)
store

stored donneau-de-vise-vengeance-des-marquis (551/1560)
stored donneau-de-vise-veuve-a-la-mode (552/1560)
stored donneau-de-vise-zelinde (553/1560)
stored dorat-celibataire (554/1560)
stored dorat-regulus (555/1560)
stored dorimond-amours-de-trapolin (556/1560)
stored dorimond-comedie-de-la-comedie (557/1560)
stored dorimond-ecole-des-cocus (558/1560)
stored dorimond-femme-industrieuse (559/1560)
stored dorneval-fuzelier-lesage-arlequin-roi-des-ogres (560/1560)
stored dorneval-jugement-de-paris (561/1560)
stored dorneval-lesage-amours-de-nanterre (562/1560)
stored dorneval-lesage-couplets-en-proces (563/1560)
stored dorneval-lesage-opera-comique (564/1560)
stored dorneval-lesage-reine-de-barostan (565/1560)
stored lesage-dorneval-ile-gougou (566/1560)
stored archambault-comedie-a-l-impromptu (567/1560)
stored archambault-on-fait-ce-qu-on-peut (568/1560)
stored dorvo-lamentin (569/1560)
stored dubosc-balance-d-etat (570/1560)
stored du-ryer-alcimedon (571/1560)
stored du-ryer-alcionee (

...

stored hartleben-hanna-jagert (188/591)
stored hartleben-die-sittliche-forderung (189/591)
stored hasenclever-der-sohn (190/591)
stored hauptmann-carl-tobias-buntschuh (191/591)
stored hauptmann-carl-im-goldenen-tempel-buche-verzeichnet (192/591)
stored hauptmann-carl-frau-nadja-bielew (193/591)
stored hauptmann-carl-musik (194/591)
stored hauptmann-carl-gaukler-tod-und-juwelier (195/591)
stored hauptmann-carl-ephraims-breite (196/591)
stored hauptmann-carl-die-armseligen-besenbinder (197/591)
stored hauptmann-carl-der-antiquar (198/591)
stored hauptmann-vor-sonnenaufgang (199/591)
stored hauptmann-gabriel-schillings-flucht (200/591)
stored hauptmann-florian-geyer (201/591)
stored hauptmann-die-ratten (202/591)
stored hebbel-trauerspiel-in-sizilien (203/591)
stored hebbel-siegfrieds-tod (204/591)
stored hebbel-maria-magdalene (205/591)
stored hebbel-kriemhilds-rache (206/591)
stored hebbel-judith (207/591)
stored hebbel-herodes-und-mariamne (208/591)
stored hebbel-gyges-und-sein-ring (

stored muellner-koenig-yngurd (378/591)
stored muellner-die-schuld (379/591)
stored muellner-der-neun-und-zwanzigste-februar (380/591)
stored nestroy-zu-ebener-erde-und-erster-stock (381/591)
stored nestroy-judith-und-holofernes (382/591)
stored nestroy-freiheit-in-kraehwinkel (383/591)
stored nestroy-eulenspiegel (384/591)
stored nestroy-einen-jux-will-er-sich-machen (385/591)
stored nestroy-der-zerrissene (386/591)
stored nestroy-der-unbedeutende (387/591)
stored nestroy-der-talisman (388/591)
stored nestroy-der-boese-geist-lumpazivagabundus (389/591)
stored nestroy-das-maedl-aus-der-vorstadt (390/591)
stored nestroy-das-haus-der-temperamente (391/591)
stored neuber-ein-deutsches-vorspiel (392/591)
stored neuber-die-verehrung-der-vollkommenheit (393/591)
stored neuber-die-beschuetzte-schauspielkunst (394/591)
stored neuber-das-schaeferfest (395/591)
stored niebergall-datterich (396/591)
stored panizza-nero (397/591)
stored panizza-das-liebeskonzil (398/591)
stored pfeil-lucie-woodvil

stored weidmann-johann-faust (563/591)
stored weidmann-der-dorfbarbier (564/591)
stored weisse-die-jagd (565/591)
stored weisse-atreus-und-thyest (566/591)
stored weissenthurn-welcher-ist-der-braeutigam (567/591)
stored weissenthurn-welche-ist-die-braut (568/591)
stored weissenthurn-johann-herzog-von-finnland (569/591)
stored weissenthurn-die-schwestern-st-janvier (570/591)
stored weissenthurn-das-manuscript (571/591)
stored werner-der-vierundzwanzigste-februar (572/591)
stored wette-haensel-und-gretel (573/591)
stored widmann-maikaefer-komoedie (574/591)
stored widmann-der-widerspenstigen-zaehmung (575/591)
stored wieland-klementina-von-porretta (576/591)
stored wieland-alceste (577/591)
stored wieland-lady-johanna-gray (578/591)
stored wilbrandt-der-meister-von-palmyra (579/591)
stored wilbrandt-gracchus-der-volkstribun (580/591)
stored wildenbruch-die-haubenlerche (581/591)
stored wildenbruch-die-karolinger (582/591)
stored wildenbruch-die-quitzows (583/591)
stored wildgans-armut (5

stored martelli-tullia (97/139)
stored de-medici-rappresentazione-di-san-giovanni-e-paolo (98/139)
stored metastasio-zenobia (99/139)
stored metastasio-temistocle (100/139)
stored metastasio-siroe (101/139)
stored metastasio-ruggero-ovvero-l-eroica-gratitudine (102/139)
stored metastasio-romolo-ed-ersilia (103/139)
stored metastasio-nitteti (104/139)
stored metastasio-l-olimpiade (105/139)
stored metastasio-l-impresario-delle-canarie (106/139)
stored metastasio-l-eroe-cinese (107/139)
stored metastasio-issipile (108/139)
stored metastasio-ipermestra (109/139)
stored metastasio-il-re-pastore (110/139)
stored metastasio-ezio (111/139)
stored metastasio-didone-abbandonata (112/139)
stored metastasio-demofoonte (113/139)
stored metastasio-demetrio (114/139)
stored metastasio-attilio-regolo (115/139)
stored metastasio-alessandro-nell-indie (116/139)
stored metastasio-adriano-in-siria (117/139)
stored metastasio-achille-in-sciro (118/139)
stored monti-caio-gracco (119/139)
stored pagano-gli-

stored plavilshchikov-ermak-pokoritel-sibiri (133/212)
stored prutkov-srodstvo-mirovyh-sil (134/212)
stored prutkov-spor-drevnih-grecheskih-filosofov (135/212)
stored prutkov-oprometchivyj-turka (136/212)
stored prutkov-fantaziya (137/212)
stored prutkov-chereposlov-sirech-frenolog (138/212)
stored prutkov-blondy (139/212)
stored pushkin-stseny-iz-rytsarskih-vremen (140/212)
stored pushkin-skupoj-rytsar (141/212)
stored pushkin-rusalka (142/212)
stored pushkin-pir-vo-vremja-chumy (143/212)
stored pushkin-mocart-i-saleri (144/212)
stored pushkin-kamenniy-gost (145/212)
stored pushkin-boris-godunov (146/212)
stored rostopchina-neludimka (147/212)
stored saltykov-shchedrin-smert-pazuhina (148/212)
stored sollogub-chinovnik (149/212)
stored sudovshchikov-neslyhannoe-divo (150/212)
stored sumarokov-vzdorschitsa (151/212)
stored sumarokov-tresotinius (152/212)
stored sumarokov-ssora-u-muzha-s-zhenoj (153/212)
stored sumarokov-sinav-i-truvor (154/212)
stored sumarokov-semira (155/212)
stored 

Test if starting and stoping works:

```
^CGracefully stopping... (press Ctrl+C again to force)
Killing vebidracor_frontend_1  ... done
Killing vebidracor_api_1       ... done
Killing vebidracor_fuseki_1    ... done
Killing vebidracor_metrics_1   ... done
ERROR: 2
```

`docker-compose -f docker-compose.empty.yml up`
works but reindexing is necessary, it seems:
```
api_1       | 04 Aug 2022 10:51:26,250 [main] INFO  (NativeBroker.java [repair]:3681) - Reindexing database files ... ```

# Freezing the container
`docker ps` shows the currently running containers

In [13]:
!docker ps | grep vebidracor

643003f3d9b8   ingoboerner/dracor-frontend:v1.4.3_local   "/docker-entrypoint.…"   4 hours ago   Up 26 minutes             0.0.0.0:8088->80/tcp               vebidracor_frontend_1
a93a3172688f   ingoboerner/dracor-api:v0.86.2_local       "java org.exist.star…"   4 hours ago   Up 26 minutes (healthy)   0.0.0.0:8080->8080/tcp, 8443/tcp   vebidracor_api_1
da38a5323ec2   stain/jena-fuseki                          "/docker-entrypoint.…"   4 hours ago   Up 26 minutes             0.0.0.0:3030->3030/tcp             vebidracor_fuseki_1
0653a94c2faa   ingoboerner/dracor-metrics:v1.2.1          "pipenv run hug -f m…"   4 hours ago   Up 26 minutes             0.0.0.0:8030->8030/tcp             vebidracor_metrics_1


In [14]:
#get the container id of the container to commit
vebidracor_api_container = "a93a3172688f"

In [15]:
!docker commit -m "prepared VeBiDraCor based on pre-built images, loaded corpora" $vebidracor_api_container ingoboerner/vebidracor-api:2.0.0

sha256:3c116f5733a38fb58fff55c4db3903a8e72cb44be9c9a25cd12d020387b21c89


In [17]:
#list
!docker images | grep vebidracor

ingoboerner/vebidracor-api                                    2.0.0                3c116f5733a3   About a minute ago   2.62GB


Pushed the images to Dockerhub `docker push ingoboerner/vebidracor-api:1.0.0`. Had to create the repos first on Dockerhub, e.g. https://hub.docker.com/repository/docker/ingoboerner/vebidracor-api

In [18]:
!docker push ingoboerner/vebidracor-api:2.0.0

The push refers to repository [docker.io/ingoboerner/vebidracor-api]

bbd53040: Preparing 
f474ceeb: Preparing 
384cad0d: Preparing 
ca3f4680: Preparing 
5d1e4382: Preparing 
b2c72703: Preparing 
103a12d5: Preparing 
97ac3576: Preparing 
d495bc2d: Preparing 
b4aa18e2: Preparing 
5e2ea8fe: Preparing 
2b22bc2c: Preparing 
7434cd5a: Preparing 
18505d06: Preparing 
46ba4b15: Preparing 
78547b20: Preparing 
d6d1483c: Preparing 
7610451d: Preparing 
87c5870e: Preparing 
934f40b4: Preparing 
4f05c29e: Preparing 
80a5a354: Preparing 
dbae9aea: Preparing 
d454860a: Preparing 


bd53040: Pushing  1.418GB/2.072GB/dracor-api [2K

bd53040: Pushed   2.075GB/2.072GB2.0.0: digest: sha256:83093d51bbf64273bf655b5eef336396d0dc1f7db3913b0e72527acee4bedd91 size: 5572


## Use the prepared copus
Use the docker-compose file `docker-compose.vebidracor.yml` to launch vebidracor: `docker-compose -f docker-compose.vebidracor.yml up`
It will take some time to have a running system, all the images need to be pulled from docker hub and the database will have to reindex:

`api_1       | 08 Aug 2022 12:00:59,719 [main] INFO  (NativeBroker.java [repair]:3681) - Reindexing database files ...`